In [113]:
# Model 분석

In [163]:
import pandas as pd
import numpy as np
import warnings; warnings.filterwarnings('ignore')

df3 = pd.read_csv('df.csv')
df3.head(3)

,ID,시설종류,실내/실외,종목,주소,이름,평점,투표횟수,시설종류.1,실내/실외.1,종목.1,이름.1,태깅
0,0,문화,실내,미술관,광주 동구 성촌길 6,드영미술관,3.0,1,문화,실내,미술관,드영미술관,문화 미술관 드영미술관
1,1,문화,실내,미술관,광주 남구 제중로47번길 10,이이남 스튜디오,4.7,84,문화,실내,미술관,이이남 스튜디오,문화 미술관 이이남 스튜디오
2,2,문화,실내,미술관,광주 북구 하서로 52,광주시립미술관,4.3,815,문화,실내,미술관,광주시립미술관,문화 미술관 광주시립미술관


In [125]:
df3.shape
df3.head(2).T

,0,1
ID,0,1
시설종류,문화,문화
실내/실외,실내,실내
종목,미술관,미술관
주소,광주 동구 성촌길 6,광주 남구 제중로47번길 10
이름,드영미술관,이이남 스튜디오
평점,3.0,4.7
투표횟수,1,84
시설종류.1,문화,문화
실내/실외.1,실내,실내


In [126]:
# id, 이름, 시설종류, 평점, 투표횟수, 
# 실내/실외, 종목
df_test = df3[ ['ID','이름', '시설종류', '평점', '투표횟수','실내/실외', '종목', '태깅'] ]
df_test.head(1).T

,0
ID,0
이름,드영미술관
시설종류,문화
평점,3.0
투표횟수,1
실내/실외,실내
종목,미술관
태깅,문화 미술관 드영미술관


In [127]:
df_test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 199 entries, 0 to 198
Data columns (total 8 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   ID      199 non-null    int64  
 1   이름      199 non-null    object 
 2   시설종류    199 non-null    object 
 3   평점      199 non-null    float64
 4   투표횟수    199 non-null    int64  
 5   실내/실외   199 non-null    object 
 6   종목      199 non-null    object 
 7   태깅      199 non-null    object 
dtypes: float64(1), int64(2), object(5)
memory usage: 12.6+ KB


In [131]:
from sklearn.feature_extraction.text import CountVectorizer

# CountVectorizer를 적용하기 위해 공백문자로 word 단위가 구분되는 문자열로 변환. 
# df_test['종목_정확도'] = df_test['종목'].apply(lambda x : (' ').join( x ))
count_vect = CountVectorizer(min_df=0, ngram_range=(1, 2))
genre_mat = count_vect.fit_transform( df_test['태깅'] )
genre_mat.shape

(199, 489)

In [132]:
# print( genre_mat[:1] )
genre_mat[:1].toarray()

array([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 

In [133]:
# 코사인 유사도 계산계산
# 반환된 코사인 유사도 행렬의 크기 및 앞 2개 데이터만 추출
from sklearn.metrics.pairwise import cosine_similarity

genre_sim = cosine_similarity(genre_mat, genre_mat)
genre_sim.shape

(199, 199)

In [134]:
genre_sim[:2]

array([[1.        , 0.50709255, 0.6       , 0.6       , 0.50709255,
        0.6       , 0.50709255, 0.6       , 0.6       , 0.6       ,
        0.6       , 0.6       , 0.59628479, 0.50709255, 0.6       ,
        0.6       , 0.6       , 0.6       , 0.6       , 0.6       ,
        0.6       , 0.2       , 0.1490712 , 0.2       , 0.16903085,
        0.2       , 0.2       , 0.2       , 0.1490712 , 0.2       ,
        0.2       , 0.16903085, 0.2       , 0.2       , 0.2       ,
        0.2       , 0.2       , 0.2       , 0.1490712 , 0.2       ,
        0.2       , 0.2       , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.2       , 0.2       , 0.2       , 0.2 

In [135]:
# 유사도가 높은 순으로 정리된 genre_sim 객체의 비교 행 위치 인덱스 값
# 값이 높은 순으로 정렬된 비교 대상 행의 유사도 값이 아니라
# 비교 대상 행의 위치 인덱스임에 주의
genre_sim_sorted_ind = genre_sim.argsort()[:, ::-1]
genre_sim_sorted_ind.shape

(199, 199)

In [136]:
df_test[['이름', '평점', '투표횟수']].sort_values(
    '평점', ascending=False)

,이름,평점,투표횟수
61,약사암,5.0,1
26,백범기념관,5.0,3
120,테스팅노트,4.9,11
138,100족발본점,4.8,187
1,이이남 스튜디오,4.7,84
...,...,...,...
101,다과만들기 다담,0.0,0
13,유ㆍ스퀘어문화관 금호갤러리,0.0,0
28,전남대학교 박물관,0.0,0
142,광후장어,0.0,0


In [137]:
C = df_test['평점'].mean()
m = df_test['투표횟수'].quantile(0.6)
C, m

(3.7306532663316587, 141.0)

In [138]:
# 기존 평점을 가중 평점으로 변경하는 함수
def weighted_vote_average(record):
  v = record['투표횟수']
  R = record['평점']
  # (예정)날씨 관련 수식을 추가 => 실내외 구분시 활용
  return ( ( (v/(v+m)) * R ) + ( (m/(v+m)) * C ) ) *2

df_test['추천점수'] = df_test.apply(weighted_vote_average, axis=1)

In [145]:
# 새롭게 부여된 가중치 평점이 높은 순으로 상위 10개의 시설
df_test[['이름', '평점', '투표횟수', '추천점수']].sort_values(
    '추천점수', ascending=False).head(10)

,이름,평점,투표횟수,추천점수
52,5·18민주화운동기록관,4.6,636,8.884484
184,쌍암공원,4.5,1119,8.827813
192,무등산 국립공원,4.5,1076,8.821729
55,5·18민주광장,4.6,488,8.810245
191,중외공원,4.5,640,8.722208
189,풍암호수공원,4.4,1734,8.699330
99,국립광주과학관,4.4,1644,8.694254
53,전일빌딩245,4.7,240,8.682531
138,100족발본점,4.8,187,8.680623
195,ACC 하늘마당,4.5,505,8.664155


In [160]:
# a = 온도 , b = 습도 , c = 1시간당 강수량

def weather_choice(a,b,c):
  y = b - ((-4.3 * a) + 147)
  if y >= 10:
    a = a + (y/10)

  if c >= 5 or a<=10 or a>=30:
    return '실내'
  else:
    return '실외'

In [158]:
# 새롭게 정의된 평점 기준에 따라 기존 find_sim_experience 함수를 변경
def find_sim_experience(df_test, sorted_ind, title_name, top_n=10):
  title_exp = df_test[df_test['이름'] == title_name]
  title_index = title_exp.index.values

  sim_index = sorted_ind[title_index, :top_n*3]
  sim_index = sim_index.reshape(-1)

  sim_index = sim_index[sim_index != title_index]

  return df_test.iloc[sim_index].sort_values(
    '추천점수', ascending=False)[:top_n]


sim_exp = find_sim_experience(df_test, genre_sim_sorted_ind, '용진정사')
test = sim_exp[['이름', '종목', '실내/실외', '평점', '투표횟수', '추천점수']]
test

,이름,종목,실내/실외,평점,투표횟수,추천점수
65,무각사,종교,실내,4.4,354,8.418675
182,광주월드컵경기장,문화,실외,4.3,597,8.382445
78,포충사,문화,실외,4.4,268,8.338494
69,원효사,종교,실내,4.3,411,8.309138
181,염주체육관,문화,실내,4.3,278,8.216812
82,월봉서원(빙월당),문화,실외,4.5,117,8.159086
183,광주 실내빙상장,문화,실내,4.2,286,8.090033
90,무등산 주상절리대,문화,실외,4.5,88,8.052595
85,만귀정,문화,실외,4.3,65,7.820603
81,호가정,문화,실외,4.1,82,7.732934


In [159]:
test[test['실내/실외'] == "실내"]

,이름,종목,실내/실외,평점,투표횟수,추천점수
65,무각사,종교,실내,4.4,354,8.418675
69,원효사,종교,실내,4.3,411,8.309138
181,염주체육관,문화,실내,4.3,278,8.216812
183,광주 실내빙상장,문화,실내,4.2,286,8.090033


In [161]:
# 날씨로 실내/실외 필터링

if weather_choice(a, b, c) == "실내":
    print(test[test['실내/실외'] == "실내"])
else :
    print(test)

NameError: name 'a' is not defined

In [162]:
import requests

url = 'http://apis.data.go.kr/1360000/VilageFcstInfoService_2.0/getUltraSrtFcst'
params ={'serviceKey' : 'OsnExmgat6fBnPcmy5/u/Ish125BWuL2gaRjXIUpbldSNUyz6obA1tlae77HEiPh67zsMKlW3nJ5MS4G/6G7jA==', 'pageNo' : '1', 'numOfRows' : '10', 'dataType' : 'XML', 'base_date' : '20211010', 'base_time' : '0630', 'nx' : '55', 'ny' : '127' }

response = requests.get(url, params=params)
print(response.content)

b'<?xml version="1.0" encoding="UTF-8"?>\r\n<response><header><resultCode>10</resultCode><resultMsg>\xec\xb5\x9c\xea\xb7\xbc 1\xec\x9d\xbc \xea\xb0\x84\xec\x9d\x98 \xec\x9e\x90\xeb\xa3\x8c\xeb\xa7\x8c \xec\xa0\x9c\xea\xb3\xb5\xed\x95\xa9\xeb\x8b\x88\xeb\x8b\xa4.</resultMsg></header></response>\r\n'


In [ ]:
# # 새롭게 정의된 평점 기준에 따라 기존 find_sim_experience 함수를 변경
# def find_sim_experience(df_test, sorted_ind, title_name, top_n=5):
#   title_exp = df_test[df_test['이름'] == title_name]
#   title_index = title_exp.index.values

#   sim_index = sorted_ind[title_index, :top_n*3]
#   sim_index = sim_index.reshape(-1)

#   sim_index = sim_index[sim_index != title_index]

#   return df_test.iloc[sim_index].sort_values(
#     '추천점수', ascending=False)[:top_n]

# name = str(input())
# sim_exp = find_sim_experience(df_test, genre_sim_sorted_ind, name)
# sim_exp[['이름', '종목', '실내/실외', '평점', '투표횟수', '추천점수']]